Import

In [1]:
import os
import openai
import sys

sys.path.append('../..')

from dotenv import load_dotenv, find_dotenv

env_path = find_dotenv()
print(f"Chemin du fichier .env trouvé : {env_path}") 


_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.environ['OPENAI_API_KEY']

Chemin du fichier .env trouvé : /home/guillaume/Code/forestbot/.env


Document

In [2]:
from langchain.document_loaders import PyPDFLoader
from langchain.schema import Document

def load_pdf_with_metadata(file_path):
    loader = PyPDFLoader(file_path)
    documents = loader.load()
    # Assign filename to each document's metadata
    for document in documents:
        document.metadata['filename'] = file_path
    return documents

# Charger le premier fichier PDF avec métadonnées
docs1 = load_pdf_with_metadata("../pdf/EdAP 2020_EN.pdf")

# Charger le deuxième fichier PDF avec métadonnées
docs2 = load_pdf_with_metadata("../pdf/SOF book-web-rev3d-hires.pdf")

# Combiner les deux listes de documents
documents = docs1 + docs2

# Afficher les métadonnées pour vérification
for document in documents:
    print(f"Document from: {document.metadata['filename']}")


Document from: ../pdf/EdAP 2020_EN.pdf
Document from: ../pdf/EdAP 2020_EN.pdf
Document from: ../pdf/EdAP 2020_EN.pdf
Document from: ../pdf/EdAP 2020_EN.pdf
Document from: ../pdf/EdAP 2020_EN.pdf
Document from: ../pdf/EdAP 2020_EN.pdf
Document from: ../pdf/EdAP 2020_EN.pdf
Document from: ../pdf/EdAP 2020_EN.pdf
Document from: ../pdf/EdAP 2020_EN.pdf
Document from: ../pdf/EdAP 2020_EN.pdf
Document from: ../pdf/EdAP 2020_EN.pdf
Document from: ../pdf/EdAP 2020_EN.pdf
Document from: ../pdf/EdAP 2020_EN.pdf
Document from: ../pdf/EdAP 2020_EN.pdf
Document from: ../pdf/EdAP 2020_EN.pdf
Document from: ../pdf/EdAP 2020_EN.pdf
Document from: ../pdf/EdAP 2020_EN.pdf
Document from: ../pdf/EdAP 2020_EN.pdf
Document from: ../pdf/EdAP 2020_EN.pdf
Document from: ../pdf/EdAP 2020_EN.pdf
Document from: ../pdf/EdAP 2020_EN.pdf
Document from: ../pdf/EdAP 2020_EN.pdf
Document from: ../pdf/EdAP 2020_EN.pdf
Document from: ../pdf/EdAP 2020_EN.pdf
Document from: ../pdf/EdAP 2020_EN.pdf
Document from: ../pdf/EdA

Generate data set

In [10]:
import time
from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from tenacity import retry, wait_exponential, stop_after_attempt, before_sleep_log
import logging

# Configuration du logger pour le backoff
logging.basicConfig()
logger = logging.getLogger("tenacity")

# Définir les modèles avec des limites de gestion
generator_llm = ChatOpenAI(model="gpt-3.5-turbo")
critic_llm = ChatOpenAI(model="gpt-4")
embeddings = OpenAIEmbeddings()

generator = TestsetGenerator.from_langchain(
    generator_llm,
    critic_llm,
    embeddings
)

# Fonction de génération avec gestion des limites de taux
@retry(wait=wait_exponential(multiplier=1, min=4, max=10), 
       stop=stop_after_attempt(10), 
       before_sleep=before_sleep_log(logger, logging.INFO))
def generate_testset(documents, test_size, distributions):
    return generator.generate_with_langchain_docs(documents, test_size=test_size, distributions=distributions)

# Appel sécurisé de la fonction de génération
try:
    testset = generate_testset(documents[0:49], test_size=10, distributions={simple: 0.5, reasoning: 0.25, multi_context: 0.25})
except Exception as e:
    print(f"Échec de la génération après plusieurs tentatives : {str(e)}")


embedding nodes:   0%|          | 0/114 [00:00<?, ?it/s]

Generating:   0%|          | 0/10 [00:00<?, ?it/s]

In [12]:
testset.to_pandas()

,question,contexts,ground_truth,evolution_type,metadata,episode_done
0,How do the Aichi Targets promote coordinated a...,[ are promoted through the Aichi Targets. They...,The Aichi Targets promote coordinated actions ...,simple,"[{'source': '../pdf/EdAP 2020_EN.pdf', 'page':...",True
1,What is the purpose of the Observatory of Biod...,[MINFOF: Ministère des Forêts et de la Faune (...,The purpose of the Observatory of Biodiversity...,simple,"[{'source': '../pdf/EdAP 2020_EN.pdf', 'page':...",True
2,How do protected areas in Central African coun...,"[40\nVarious COMIFAC directives, notably those...",Protected areas in Central African countries c...,simple,"[{'source': '../pdf/EdAP 2020_EN.pdf', 'page':...",True
3,How does habitat destruction pose a threat to ...,[32\n2.2. Rich but threatened \nanimal divers...,Habitat destruction poses a threat to great ap...,simple,"[{'source': '../pdf/EdAP 2020_EN.pdf', 'page':...",True
4,What were the broad objectives for the overhau...,[42\nand to clarify its positioning in the nat...,The broad objectives for the overhaul of UTOs ...,simple,"[{'source': '../pdf/EdAP 2020_EN.pdf', 'page':...",True
5,What actions are needed for Aichi Targets to a...,[ are promoted through the Aichi Targets. They...,Coordinated actions at different scales and be...,reasoning,"[{'source': '../pdf/EdAP 2020_EN.pdf', 'page':...",True
6,How do certified forestry concessions help eco...,[29all territories and all socio-economic act...,Certified forestry concessions under sustainab...,reasoning,"[{'source': '../pdf/EdAP 2020_EN.pdf', 'page':...",True
7,How do forest concessions in the DRC help biod...,[28\nforest cover for the conservation of frag...,Forest concessions in the DRC help biodiversit...,multi_context,"[{'source': '../pdf/EdAP 2020_EN.pdf', 'page':...",True
8,What is the main goal of SOPA reports and how ...,"[14\nPREFACE\nF. Palla, L. Itsoua-Madzous & C....",The main goal of SOPA reports is to contribute...,multi_context,"[{'source': '../pdf/EdAP 2020_EN.pdf', 'page':...",True
9,How do Central African protected areas support...,"[19rainfall, supporting agricultural systems a...",Central African protected areas contribute to ...,multi_context,"[{'source': '../pdf/EdAP 2020_EN.pdf', 'page':...",True


Save dataset

In [28]:
import pandas as pd
df_testset = testset.to_pandas()
df_testset.to_csv('evaluation_dataset.csv', index=False)  # `index=False` to not include label in the file


In [29]:
print(df_testset.columns)
print(df_testset.head)

Index(['question', 'contexts', 'ground_truth', 'evolution_type', 'metadata',
       'episode_done'],
      dtype='object')
<bound method NDFrame.head of                                             question  \
0  How do the Aichi Targets promote coordinated a...   
1  What is the purpose of the Observatory of Biod...   
2  How do protected areas in Central African coun...   
3  How does habitat destruction pose a threat to ...   
4  What were the broad objectives for the overhau...   
5  What actions are needed for Aichi Targets to a...   
6  How do certified forestry concessions help eco...   
7  How do forest concessions in the DRC help biod...   
8  What is the main goal of SOPA reports and how ...   
9  How do Central African protected areas support...   

                                            contexts  \
0  [ are promoted through the Aichi Targets. They...   
1  [MINFOF: Ministère des Forêts et de la Faune (...   
2  [40\nVarious COMIFAC directives, notably those...   
3  [32